In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataLoad import PulsarData
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

In [2]:
# For pretty plotting
plt.style.use('seaborn-paper')
plt.rcParams["font.family"] = "serif"

Creating the neural network:

In [3]:
class NeuralN(nn.Module):
    def __init__(self,inputsize,hiddensize):
        super(NeuralN, self).__init__()
        self.inputsize=inputsize
        self.hiddensize=hiddensize
        # an affine operation: y = Wx + b, this is basically a weight tensor!
        self.fcinput = nn.Linear(in_features=self.inputsize, out_features=self.hiddensize)
        self.fcoutput = nn.Linear(in_features=self.hiddensize, out_features=2)
    
    def forward(self,x: torch.Tensor):
        x = self.fcinput(x)
        x = F.relu(x)
        x = self.fcoutput(x)
        return x

Loading the data:

In [5]:
raw_features = PulsarData('HTRU_2').features
raw_targets = PulsarData('HTRU_2').targets

Defining the epochs for the neural network to train:

In [6]:
epochs = 10

Splitting data into test and train data:

In [7]:
train_features_data, test_features_data, train_targets_data, test_targets_data  =  train_test_split( raw_features, 
                                                        raw_targets, test_size=0.25, random_state=42)

Writing a cross validation function that is compatible with torch: 

In [8]:
def NN_CrossValidation(hiddensize, learning_rate, data, targets):
   cv = 5
   net = NeuralN(data.shape[1], hiddensize)
   dlist = np.array_split(data.to_numpy(), cv)
   tlist = np.array_split(targets.to_numpy(), cv)
   cval = list()
   for d, dat in tqdm(enumerate(dlist)):
      cross_dlist =  dlist[:d] + dlist[d+1 :]
      cross_tlist =  tlist[:d] + tlist[d+1 :]
      cross_dat = torch.from_numpy(np.concatenate(cross_dlist)).float()
      cross_tar = torch.from_numpy(np.concatenate(cross_tlist)).long()
      criterion = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
      net.train()
      for e in range(epochs):
         epoch_losses = list()
         for n in range(cross_dat.shape[0]):
            net.zero_grad()
            optimizer.zero_grad() 
            prediction = net(cross_dat[n]).unsqueeze(0)
            target = cross_tar[n].unsqueeze(0)
            # Calculating the loss function
            loss = criterion(prediction,target)
            epoch_losses.append(float(loss))
            # Calculating the gradient
            loss.backward()
            optimizer.step()
      net.eval()
      cross_pred = torch.argmax(net(torch.from_numpy(dat).float()),dim=1)
      acc_cross = torch.mean((cross_pred == torch.from_numpy(tlist[d]).long()).float())
      cval.append(acc_cross)

   return np.mean(np.array(cval))

In [9]:
def optimize_NN(data, targets, pars, n_iter=5):
    """Apply Bayesian Optimization to Neural Network parameters."""
    
    def crossval_wrapper(hiddensize, learning_rate):
        """Wrapper of Neural Network cross validation. 
           Notice how we ensure params are casted to integer before we pass them along.
        """
        return NN_CrossValidation(hiddensize=int(hiddensize), 
                                            learning_rate=learning_rate, 
                                            data=data, 
                                            targets=targets)

    boptimizer = BayesianOptimization(f=crossval_wrapper, 
                                     pbounds=pars, 
                                     random_state=42, 
                                     verbose=2)
    boptimizer.maximize(init_points=4, n_iter=n_iter)

    return boptimizer


In [10]:
parameters_BayesianOptimization = {"hiddensize": (1, 50), 
                                   "learning_rate": (0.00001, 0.05),
                                  }

BayesianOptimization = optimize_NN(train_features_data, 
                                             train_targets_data, 
                                             parameters_BayesianOptimization, 
                                             n_iter=5)
print(BayesianOptimization.max)

  0%|          | 0/2 [00:00<?, ?it/s]|   iter    |  target   | hidden... | learni... |
-------------------------------------------------
100%|██████████| 2/2 [00:10<00:00,  5.04s/it]
|  4        |  0.9081   |  3.846    |  0.4331   |
100%|██████████| 2/2 [00:10<00:00,  5.15s/it]
|  6        |  0.9569   |  28.06    |  0.04473  |
100%|██████████| 2/2 [00:10<00:00,  5.23s/it]
|  7        |  0.9081   |  28.99    |  0.4956   |
100%|██████████| 2/2 [00:10<00:00,  5.35s/it]
|  8        |  0.9081   |  1.0      |  1e-05    |
100%|██████████| 2/2 [00:11<00:00,  5.51s/it]|  9        |  0.9684   |  44.03    |  1e-05    |
{'target': 0.9684126973152161, 'params': {'hiddensize': 44.02696538592287, 'learning_rate': 1e-05}}



Creating a neural network with the optimal hiddensize:

In [11]:
net = NeuralN(raw_features.shape[1], int(BayesianOptimization.max['params']['hiddensize']))

Converting data into torch tensors:

In [12]:
train_features_data, test_features_data = torch.from_numpy(train_features_data.to_numpy()).float(), torch.from_numpy(test_features_data.to_numpy()).float()
train_targets_data, test_targets_data = torch.from_numpy(train_targets_data.to_numpy()).long(), torch.from_numpy(test_targets_data.to_numpy()).long()

Setting the optimal learning rate and training the network:

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=BayesianOptimization.max['params']['learning_rate'])
net.train()

NeuralN(
  (fcinput): Linear(in_features=8, out_features=44, bias=True)
  (fcoutput): Linear(in_features=44, out_features=2, bias=True)
)

In [14]:
epochs = 10
for e in range(epochs):
    epoch_losses = list()
    for n in range(train_features_data.shape[0]):
        net.zero_grad()
        optimizer.zero_grad() 
        prediction = net(train_features_data[n]).unsqueeze(0)
        target = train_targets_data[n].unsqueeze(0)
        # Calculating the loss function
        loss = criterion(prediction,target)
        epoch_losses.append(float(loss))
        # Calculating the gradient
        loss.backward()
        optimizer.step()
    print(e, np.mean(epoch_losses))

net.eval()

0 0.7955809585704927
1 0.11341940348229061
2 0.09552984293327933
3 0.09223155197848301
4 0.09053195488000564
5 0.08938073620988715
6 0.08853963015316019
7 0.08788191352316818
8 0.08734193770000165
9 0.08688318341871432


NeuralN(
  (fcinput): Linear(in_features=8, out_features=44, bias=True)
  (fcoutput): Linear(in_features=44, out_features=2, bias=True)
)

Final result for train data and test data:

In [15]:
train_prediction = torch.argmax(net(train_features_data),dim=1)
acc_train = torch.mean((train_prediction == train_targets_data).float())
test_prediction = torch.argmax(net(test_features_data),dim=1)
acc_test = torch.mean((test_prediction == test_targets_data).float())

print(acc_train, acc_test)

tensor(0.9747) tensor(0.9736)
